In [18]:
import os
from dotenv import load_dotenv, dotenv_values
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.agents import AgentExecutor, Tool,initialize_agent
from langchain.agents.types import AgentType
load_dotenv()

True

In [169]:
system_message = """
"You are the XYZ bot."
"This is conversation with a human. Answer the questions you get based on the knowledge you have."
"If you don't know the answer, just say that you don't, don't try to make up an answer."
"""

In [21]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",  # Name of the language model
    temperature=0  # Parameter that controls the randomness of the generated responses
)

In [22]:
embeddings = OpenAIEmbeddings()

c:\Users\victo\Desktop\data\data-science-snippets\env-data\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [126]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader, TextLoader

from langchain_community.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("docs")

doc = loader.load()


In [143]:
print(doc[17].metadata['source'], doc[17].metadata['page'],'\n')
print(doc[17].page_content)


docs\download.pdf 2 

i. Kenya Power & Lighting Company Limited & another v Zakayo Saitoti Naingola & another
(2008) eKLR
ii. HB (minor suing through mother & next friend DKM) v jasper Nchonga Magari & another
(2021) eKLR
iii. Eva Karemi & 5 others v Koskei Kieng & another (2020) eKLR
iv. Homabay Civil Appeal No. E113 of 2021-James Kwanya Rege v Loice Mbone Cweya.
14. The appellant submitted that costs followed the event and prayed for costs of this Appeal based on
section 27(1) of the Civil Procedure Act.
The respondent’s Submissions.
15. Through her submissions dated March 24, 2023, the respondent submitted that Dr. Peter Morebu
opined that she sustained multiple injuries which were in the process of healing and that she needed
analgesics when she was in pain. That the award of Kshs 200,000/= was not too high to warrant
interference by this court. She relied on Charles Gichuki v Emily Kawira Mbuba & another (2018)
eKLR.
16. It was the respondents’ submission that the award by the tri

In [144]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators= "\n",
    chunk_size = 1000,
    chunk_overlap = 200,
)
text_chunks = text_splitter.split_documents(doc)

In [146]:
len(text_chunks)

81

In [145]:
vectorstore = FAISS.from_documents(text_chunks, embeddings)

In [147]:
retriever = vectorstore.as_retriever()

In [150]:
docs = retriever.get_relevant_documents("car accident with brocken femur")

In [159]:
docs[1].metadata, docs[1].page_content

({'source': 'docs\\Civil_Appeal_52_of_2020.pdf', 'page': 1},
 '9.  It was PW1’s further testimony that there was a metal plate in his leg that needed to be removed for Kshs.180,000/=. He denied\ngoing to the road or using his mobile phone. He told the court that he was walking along a pedestrian path  when he was knocked by\nthis vehicle and that he totally blamed the 1st Defendant for the accident.\n10.  On cross-examination PW1 testified that he was not walking along the zebra crossing, but along the machine lines. He\nmaintained that he was a driver who used to earn Kshs.50,000/=.\n11. PW2 was Dr. Ajoni Adede  who examined PW1 on the 21st November, 2018.  He stated that PW1 was involved in a road\ntraffic accident on 8th November, 201 8 and suffered fractures of the right tibia and fibula leg bones as well as a dislocation of the\nright ankle and a bruise on the right leg. He testified that he examined PW1 thirteen days after the accident.')

In [97]:
memory = ConversationBufferMemory(memory_key="chat_history", input_key='input', return_messages=True, output_key='output')

In [161]:
qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        verbose=True,
        return_source_documents=True
    )

In [162]:
tools = [
        Tool(
            name="doc_search_tool",
            func=qa,
            description=(
               "This tool is used to retrieve information from the knowledge base"
            )
        )
    ]

In [170]:
agent = initialize_agent(
        agent = AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
        tools=tools,
        llm=llm,
        memory=memory,
        return_source_documents=True,
        return_intermediate_steps=True,
        agent_kwargs={"system_message": system_message}
        )

In [178]:
query1 = "based on the cases return iformation relevant to broken femur including there page numbers"
result1 = agent(query1)



> Entering new RetrievalQA chain...

> Finished chain.


In [183]:
result1['intermediate_steps'][0][1]['source_documents'][0].metadata

{'source': 'docs\\Civil_Appeal_52_of_2020.pdf', 'page': 3}

In [180]:
result1['output']

'The information provided does not mention a broken femur in any of the cases discussed. Therefore, based on the context provided, I do not have information about a broken femur in these specific legal cases.'

In [181]:
dict(result1)

{'input': 'based on the cases return iformation relevant to broken femur including there page numbers',
 'chat_history': [HumanMessage(content='how to appoint chief justice?'),
  AIMessage(content="The Chief Justice of a country is typically appointed by the head of state or government, following a process outlined in the country's constitution or laws. The specific procedure can vary between different countries."),
  HumanMessage(content='how to appoint chief justice?'),
  AIMessage(content="I don't know."),
  HumanMessage(content='my client was in a car accident with brocken femur'),
  AIMessage(content="I am sorry to hear about your client's car accident and broken femur. It's important for your client to seek medical attention immediately and consider consulting with a personal injury lawyer to understand their legal options for seeking compensation for their injuries."),
  HumanMessage(content='my client was in a car accident with brocken femur how can i win this case'),
  AIMessa

In [182]:
for x in range(len(result1['intermediate_steps'][0][1]["source_documents"][0].metadata)):
    print(result1["source_documents"][x].metadata)

KeyError: 'source_documents'